In [ ]:
#install necessary dependencies
!pip install flask pyngrok requests --quiet

#setting up ngrok authentication
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz --quiet
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok config add-authtoken 2q1QOym4phRHlm9QPGJuJfBY1zo_ye63SKnsh7bsNN6xFMBB

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#import required libraries
from flask import Flask, request, jsonify, render_template, redirect, url_for
from pyngrok import ngrok
import os
import requests
from google.colab import drive
import logging

#enabling debugging logs
logging.basicConfig(level=logging.DEBUG)

In [ ]:
#mount Google Drive
drive.mount('/content/drive')
users_file = "user_profiles.txt"
template_folder = "/content/drive/MyDrive/FlaskApp/templates"

#initialise the text file if it doesn't exist
if not os.path.exists(users_file):
    open(users_file, 'w').close()

Mounted at /content/drive


In [ ]:
#server 1 (Database Server)
server1 = Flask(__name__)

#helper functions to load, save and update user data
def load_users():
    """Load user profiles from file."""
    users = {}
    with open(users_file, 'r') as f:
        for line in f:
            email, age = line.strip().split(',')
            users[email] = {'email': email, 'age': age}
    return users

def save_user(email, age):
    """Save a user profile."""
    with open(users_file, 'a') as f:
        f.write(f"{email},{age}\n")

def update_users(users):
    """Update all user profiles."""
    with open(users_file, 'w') as f:
        for email, data in users.items():
            f.write(f"{email},{data['age']}\n")

In [ ]:
#route for server 1
@server1.route('/')
def server1_index():
    return "Welcome to the User Profile Database Server"

#routes for user management
@server1.route('/add_user', methods=['POST'])
def add_user():
    data = request.get_json()
    email = data.get('email')
    age = data.get('age')
    if not email or not age:
        return jsonify({"error": "Email and age are required!"}), 400
    users = load_users()
    if email in users:
        return jsonify({"error": "User already exists!"}), 400
    save_user(email, age)
    return jsonify({"message": "User added successfully!"}), 201

@server1.route('/get_user/<email>', methods=['GET'])
def get_user(email):
    users = load_users()
    if email not in users:
        return jsonify({"error": "User not found!"}), 404
    return jsonify(users[email]), 200

@server1.route('/update_user/<email>', methods=['PUT'])
def update_user(email):
    data = request.get_json()
    users = load_users()
    if email not in users:
        return jsonify({"error": "User not found!"}), 404
    if 'age' in data:
        users[email]['age'] = data['age']
    update_users(users)
    return jsonify({"message": "User updated successfully!"}), 200

@server1.route('/delete_user/<email>', methods=['DELETE'])
def delete_user(email):
    users = load_users()
    if email not in users:
        return jsonify({"error": "User not found!"}), 404
    del users[email]
    update_users(users)
    return jsonify({"message": "User deleted successfully!"}), 200

In [ ]:
#starting server 1 with ngrok
public_url_server1 = ngrok.connect(5000).public_url
print(f"Server 1 is running at {public_url_server1}")
server1_port = 5000

import threading
server1_thread = threading.Thread(target=server1.run, kwargs={"port": server1_port, "debug": True, "use_reloader": False})
server1_thread.start()

Server 1 is running at https://335d-34-139-171-33.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: on


In [ ]:
#server 2 (web UI server)
server2 = Flask(__name__, template_folder=template_folder)

In [ ]:
@server2.route('/')
def index():
    return render_template('index.html')

@server2.route('/add_user', methods=['POST'])
def server2_add_user():
    email = request.form['email']
    age = request.form['age']
    data = {'email': email, 'age': age}

    #forward request to server 1
    try:
        response = requests.post(f"{public_url_server1}/add_user", json=data)
        if response.status_code == 201:
            return redirect(url_for('index'))
        return f"Error: {response.json()['error']}", 400
    except requests.exceptions.InvalidSchema as e:
        print(f"Invalid Schema Error: {e}")
        return f"Error: Invalid URL schema used for request.", 500

@server2.route('/get_user/<email>', methods=['GET'])
def server2_get_user(email):
    response = requests.get(f"{public_url_server1}/get_user/{email}")
    if response.status_code == 200:
        user = response.json()
        return render_template('user.html', user=user)
    return "User not found", 404

In [ ]:
#starting server 1 with ngrok
public_url_server2 = ngrok.connect(5001).public_url
print(f"Server 2 is running at {public_url_server2}")
server2_port = 5001

server2_thread = threading.Thread(target=server2.run, kwargs={"port": server2_port, "debug": True, "use_reloader": False})
server2_thread.start()

Server 2 is running at https://c6a6-34-139-171-33.ngrok-free.app
 * Serving Flask app '__main__'
